In [1]:
import os
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [3]:
#load dataset

In [4]:
train_df=pd.read_csv('MIT-BIH arrythmia dataset with 21 record 5 class sampled dataset using R-peack header=True.csv')

In [5]:
print(train_df['target'].value_counts())

0    8600
3    6773
1    6320
2    6070
4    3985
Name: target, dtype: int64


In [6]:
y_train_=to_categorical(train_df['target'])

In [7]:
X_train_=train_df.iloc[:,:-1].values

In [8]:
X_train_.shape

(31748, 360)

In [57]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_,y_train_, test_size = 0.1)

In [45]:
X_train.shape

(28573, 360)

In [43]:
from tensorflow.keras import Sequential,utils
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPooling1D, Dropout,GlobalAveragePooling1D,AveragePooling1D


In [56]:
model=None

In [58]:

#---Model---

batch = 16
epochs = 10
shape = np.size(X_train,1)


model = Sequential()
model.add(Dense(360, activation='relu', input_shape = (shape,1)))
model.add(Conv1D(100, 10, activation='relu'))
model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu'))
model.add(Conv1D(160, 10, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.3))
model.add(Dense(5, activation='sigmoid')) 
model.summary()
model.compile(loss='binary_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])


X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)


model.fit(X_train,y_train, batch_size = batch, epochs = epochs)
score = model.evaluate(X_test, y_test, batch_size = batch)
score

y_pred = model.predict(X_test, batch_size = batch)
threshold = 0.5
y_pred1 = y_pred[:,0] < threshold



Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 360, 360)          720       
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 351, 100)          360100    
_________________________________________________________________
conv1d_74 (Conv1D)           (None, 342, 100)          100100    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 114, 100)          0         
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 105, 160)          160160    
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 96, 160)           256160    
_________________________________________________________________
global_average_pooling1d_6 ( (None, 160)             

NameError: name 'Y_test' is not defined

In [60]:
model.save('CNN1D_MODEL.h5')

In [59]:
cm = confusion_matrix(y_test, y_pred1)
cm

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets